In [1]:
import numpy as np
import pandas as pd
from Pricing.MC_simulation import simulateSABRMonteCarloEuler, simulateSABRMonteCarloMilstein, drawTwoRandomNumbers
from Pricing.black_pricing import black,dPlusBlack,dMinusBlack,find_ivol
import math

In [2]:
data=pd.read_csv('../Fitter/parameters/params_Hagan.csv')
data.set_index('Unnamed: 0',inplace=True)

factors=data.iloc[:,27].values #take swaption of 15 tenor and 10Y expiry as example

In [3]:
factors

array([  0.02312935,   0.35424604,  -0.07782868,   0.25810059,
         0.02708095,  10.        ])

In [4]:
forwards=simulateSABRMonteCarloMilstein(100000,78, 10, 0.0270809467677389, float(factors[0]),0.354,-0.07,0.258)

In [5]:
diffs=[item-(0.027080947-150.0/10000) for item in forwards]
options=[item if item>0 else 0 for item in diffs]
options=np.array(options)
p=options.mean()*math.exp(-10*2.35/100) #swaption price

In [6]:
p

0.013576936032989503

In [7]:
p*math.exp(-10*2.35/100)

0.010733529854955506

In [12]:
find_ivol(p,0.027080947,0.027080947-150.0/10000,10,0.0235)

0.2 0.0136349449181 0.0111988297703
0.21 0.0136349449181 0.0111368011764
0.22 0.0136349449181 0.0110787212724
0.23 0.0136349449181 0.0110247854605
0.24 0.0136349449181 0.0109750683059
0.25 0.0136349449181 0.0109295554855
0.26 0.0136349449181 0.0108881687623
0.27 0.0136349449181 0.0108507852058
0.28 0.0136349449181 0.0108172517999
0.29 0.0136349449181 0.0107873964245
0.3 0.0136349449181 0.0107610360218
0.31 0.0136349449181 0.0107379826001
0.32 0.0136349449181 0.0107180475889
0.33 0.0136349449181 0.0107010449439
0.34 0.0136349449181 0.010686793311
0.35 0.0136349449181 0.0106751174853
0.36 0.0136349449181 0.010665849345
0.37 0.0136349449181 0.0106588283978
0.38 0.0136349449181 0.0106539020435
0.39 0.0136349449181 0.0106509256308
0.4 0.0136349449181 0.0106497623684
0.41 0.0136349449181 0.0106502831336
0.42 0.0136349449181 0.0106523662125
0.43 0.0136349449181 0.0106558969962
0.44 0.0136349449181 0.0106607676508
0.45 0.0136349449181 0.010666876775
0.46 0.0136349449181 0.0106741290548
0.47 0.

'failture to find the right ivol!'

In [10]:
black(0.027080947,0.027080947-150.0/10000,10,0.3545,1,r=0.0235)

0.010670658651487858

In [19]:
find_ivol(-0.001518735348974044,0.027080947,0.027080947+25.0/10000,10,0)

0.2